In [10]:
import collections
from csv import DictReader

import numpy as np
import matplotlib.pyplot as plt

In [11]:
season16_17 = DictReader(open('podatki/season-1617_csv.csv', 'rt', encoding='utf-8'))
season17_18 = DictReader(open('podatki/season-1718_csv.csv', 'rt', encoding='utf-8'))
season18_19 = DictReader(open('podatki/season-1819_csv.csv', 'rt', encoding='utf-8'))

In [12]:
for row in season16_17:
    div = row["Div"]
    date = row["Date"]
    home_team = row["HomeTeam"]
    away_team = row["AwayTeam"]
    fthg = row["FTHG"]
    ftag = row["FTAG"]
    ftr = row["FTR"]
    hthg = row["HTHG"]
    htag = row["HTAG"]
    htr = row["HTR"]
    referee = row["Referee"]
    hts = row["HS"]
    ats = row["AS"]
    hst = row["HST"]
    ast = row["AST"]
    hf = row["HF"]
    af = row["AF"]
    hc = row["HC"]
    ac = row["AC"]
    hy = row["HY"]
    ay = row["AY"]
    hr = row["HR"]
    ar = row["AR"]
    print(f"{home_team}   {away_team}")
    
    
    
    

Burnley   Swansea
Crystal Palace   West Brom
Everton   Tottenham
Hull   Leicester
Man City   Sunderland
Middlesbrough   Stoke
Southampton   Watford
Arsenal   Liverpool
Bournemouth   Man United
Chelsea   West Ham
Man United   Southampton
Burnley   Liverpool
Leicester   Arsenal
Stoke   Man City
Swansea   Hull
Tottenham   Crystal Palace
Watford   Chelsea
West Brom   Everton
Sunderland   Middlesbrough
West Ham   Bournemouth
Chelsea   Burnley
Crystal Palace   Bournemouth
Everton   Stoke
Hull   Man United
Leicester   Swansea
Southampton   Sunderland
Tottenham   Liverpool
Watford   Arsenal
Man City   West Ham
West Brom   Middlesbrough
Arsenal   Southampton
Bournemouth   West Brom
Burnley   Hull
Liverpool   Leicester
Man United   Man City
Middlesbrough   Crystal Palace
Stoke   Tottenham
West Ham   Watford
Swansea   Chelsea
Sunderland   Everton
Chelsea   Liverpool
Everton   Middlesbrough
Hull   Arsenal
Leicester   Burnley
Man City   Bournemouth
West Brom   West Ham
Crystal Palace   Stoke
Southa

In [7]:
for row in season17_18:
    div = row["Div"]
    date = row["Date"]
    home_team = row["HomeTeam"]
    away_team = row["AwayTeam"]
    fthg = row["FTHG"]
    ftag = row["FTAG"]
    ftr = row["FTR"]
    hthg = row["HTHG"]
    htag = row["HTAG"]
    htr = row["HTR"]
    referee = row["Referee"]
    hts = row["HS"]
    ats = row["AS"]
    hst = row["HST"]
    ast = row["AST"]
    hf = row["HF"]
    af = row["AF"]
    hc = row["HC"]
    ac = row["AC"]
    hy = row["HY"]
    ay = row["AY"]
    hr = row["HR"]
    ar = row["AR"]
    
    

In [8]:
for row in season18_19:
    div = row["Div"]
    date = row["Date"]
    home_team = row["HomeTeam"]
    away_team = row["AwayTeam"]
    fthg = row["FTHG"]
    ftag = row["FTAG"]
    ftr = row["FTR"]
    hthg = row["HTHG"]
    htag = row["HTAG"]
    htr = row["HTR"]
    referee = row["Referee"]
    hts = row["HS"]
    ats = row["AS"]
    hst = row["HST"]
    ast = row["AST"]
    hf = row["HF"]
    af = row["AF"]
    hc = row["HC"]
    ac = row["AC"]
    hy = row["HY"]
    ay = row["AY"]
    hr = row["HR"]
    ar = row["AR"]
    